In [109]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint

In [2]:
## from http://users.telenet.be/d.rijmenants/en/enigmatech.htm
entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  #(rotor right side)   
I     = 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'
II    = 'AJDKSIRUXBLHWTMCQGZNPYFVOE'
III   = 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
IV    = 'ESOVPZJAYQUIRHXLNFTGKDCMWB'
V     = 'VZBRGITYUPSDNHLXAWMJQOFECK'
## Additional rotors used by Kriegsmarine M3 and M4 only:
VI    = 'JPGVOUMFYQBENHZRDKASXLICTW'
VII   = 'NZJHGRCXMYSWBOUFAIVLPEKQDT'
VIII  = 'FKQHTLXOCBJSPDZRAMEWNIUYGV'

## reflectors, standard army & airforce
contacts   = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
breflector = 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
creflector = 'FVPJIAOYEDRZXWGCTKUQSBNMHL'

In [3]:
raw_rotors = {'I':I, 'II':II, 'III':III, 'IV':IV, 'V':V}
print(raw_rotors)

{'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}


In [4]:
base_rotors = {'I':[], 'II':[], 'III':[], 'IV':[], 'V':[]}
for k in base_rotors.keys():
    base_rotors[k] = [ascii_uppercase.index(c) for c in raw_rotors[k]]
# rI = [ascii_uppercase.index(c) for c in I]
for k,v in base_rotors.items():
    print(k,':',v)

I : [4, 10, 12, 5, 11, 6, 3, 16, 21, 25, 13, 19, 14, 22, 24, 7, 23, 20, 18, 15, 0, 8, 1, 17, 2, 9]
II : [0, 9, 3, 10, 18, 8, 17, 20, 23, 1, 11, 7, 22, 19, 12, 2, 16, 6, 25, 13, 15, 24, 5, 21, 14, 4]
III : [1, 3, 5, 7, 9, 11, 2, 15, 17, 19, 23, 21, 25, 13, 24, 4, 8, 22, 6, 0, 10, 12, 20, 18, 16, 14]
IV : [4, 18, 14, 21, 15, 25, 9, 0, 24, 16, 20, 8, 17, 7, 23, 11, 13, 5, 19, 6, 10, 3, 2, 12, 22, 1]
V : [21, 25, 1, 17, 6, 8, 19, 24, 20, 15, 18, 3, 13, 7, 11, 23, 0, 22, 12, 9, 16, 14, 5, 4, 2, 10]


In [60]:
rev_rotors = {}

for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

for k,v in rev_rotors.items():
    print(k,':',v)

I : [20, 22, 24, 6, 0, 3, 5, 15, 21, 25, 1, 4, 2, 10, 12, 19, 7, 23, 18, 11, 17, 8, 13, 16, 14, 9]
II : [0, 9, 15, 2, 25, 22, 17, 11, 5, 1, 3, 10, 14, 19, 24, 20, 16, 6, 4, 13, 7, 23, 12, 8, 21, 18]
III : [19, 0, 6, 1, 15, 2, 18, 3, 16, 4, 20, 5, 21, 13, 25, 7, 24, 8, 23, 9, 22, 11, 17, 10, 14, 12]
IV : [7, 25, 22, 21, 0, 17, 19, 13, 11, 6, 20, 15, 23, 16, 2, 4, 9, 12, 1, 18, 10, 3, 24, 14, 8, 5]
V : [16, 2, 24, 11, 23, 22, 4, 13, 5, 19, 25, 14, 18, 12, 21, 9, 20, 3, 10, 6, 8, 0, 17, 15, 7, 1]


In [6]:
reflectors = {'Brf': {k:v for k,v in zip(entry, breflector)}, 'Crf': {k:v for k,v in zip(entry, creflector)}}
print(reflectors)

{'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}


In [66]:
def once_thru_scramble(ch, direction='forward', first_rotor='I', pos1=0, second_rotor='II', pos2=0, 
                       third_rotor='III', pos3=0):
    """ ch must be single ASCII character A-Z
    direction is either 'forward' or 'back' """
    if direction == 'forward':
        usedict = {k:v for k,v in base_rotors.items()}
    elif direction == 'back':
        usedict = {k:v for k,v in rev_rotors.items()}
    else:
        print('only forward or back for direction')
        return 'wtf'
    
    entry = ascii_uppercase
    ch = ch.upper()
    entry_pos = entry.index(ch)
    fst_pos_mod = (26 + pos1 - 0)%26
#     print(fst_pos_mod)
    fst_in = (entry_pos + fst_pos_mod)%26
#     print(fst_in)
    fst_out = usedict[first_rotor][fst_in]
#     print(fst_out)
    ch1o = entry[fst_out]
    print('first rotor:  ', ch, '-->', ch1o)
    
    scd_pos_mod = (26 + pos2 - pos1)%26
    scd_in = (fst_out + scd_pos_mod)%26
    ch2i = entry[scd_in]
    print('r1 to r2:     ', ch1o, '-->', ch2i)
#     print(scd_pos_mod, scd_in)
    scd_out = usedict[second_rotor][scd_in]
#     print(scd_out)
    ch2o = entry[scd_out]
    print('second rotor: ', ch2i, '-->', ch2o)
    
    thd_pos_mod = (26 + pos3 - pos2)%26
    thd_in = (scd_out + thd_pos_mod)%26
    ch3i = entry[thd_in]
    print('r2 to r3:     ', ch2o, '-->', ch3i)
    thd_out = usedict[third_rotor][thd_in]
#     print(thd_out)
    ch3o = entry[thd_out]
    print('third rotor:  ', ch3i, '-->', ch3o)
    
    return ch3o
    
    
once_thru_scramble('q')

first rotor:   Q --> X
r1 to r2:      X --> X
second rotor:  X --> V
r2 to r3:      V --> V
third rotor:   V --> M


'M'

In [72]:
def full_scramble(in_ch, left_rotor='III', lpos=0, middle_rotor='II', mpos=0, right_rotor='I', rpos=0, rflector='Brf'):
    entry = ascii_uppercase
    in_ch = in_ch.upper()
    
    ## first run right to left through scrambler
    forward_run = once_thru_scramble(in_ch, 'forward', first_rotor=right_rotor, pos1=rpos, second_rotor=middle_rotor,
                                     pos2=mpos, third_rotor=left_rotor, pos3=lpos)
    
    ## reflector back around for return
    rfi_pos_mod = (26 + 0 - lpos)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
    rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
    chri = entry[rf_in]
    print('r3 to reflctr:', forward_run, '-->', chri)
    mirrored = reflectors[rflector][chri]
    print('reflected:    ', chri, '-->', mirrored)
    
    ## second run back left to right thru scrambler
    back_run = once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=lpos, 
                                  second_rotor=middle_rotor, pos2=mpos, third_rotor=right_rotor, pos3=rpos)
    
    bk_out = entry.index(back_run)
    bko_pos_mod = (26 + 0 - rpos)%26  ## as above, '0' just reflects that the entry interface doesn't move
    bk_final = (bk_out + bko_pos_mod)%26
    final = entry[bk_final]
    print('r3 back out:  ', back_run, '-->', final)
    
    return final

# full_scramble('i')

In [74]:
full_scramble('q', lpos=2, mpos=22, rpos=10)

first rotor:   Q --> E
r1 to r2:      E --> Q
second rotor:  Q --> Q
r2 to r3:      Q --> W
third rotor:   W --> U
r3 to reflctr: U --> S
reflected:     S --> F
first rotor:   F --> D
r1 to r2:      D --> X
second rotor:  X --> I
r2 to r3:      I --> W
third rotor:   W --> N
r3 back out:   N --> D


'D'

In [89]:
def step_rotor(p_in):
    if p_in == 25:
        p_in = 0
    else:
        p_in += 1
    return p_in

In [105]:
word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [111]:
def enigmatise(tocode, rstartpos=0, mstartpos=0, lstartpos=0):
    encoded = ''

    tocode = tocode.replace(' ','')

    lftpos = 11
    midpos = 3
    rgtpos = 24

    mnotch = 4   ## point if right rotor reaches will trigger middle rotor to step
    lnotch = 5  ## point if middle rotor reaches will trigger left rotor to step

    for c in tocode:
        print('in:    ', c)

        rgtpos = step_rotor(rgtpos)
        if rgtpos == mnotch:
            midpos = step_rotor(midpos)
            print('--- middle rotor step ---')
            if midpos == lnotch:
                lftpos = step_rotor(lftpos)
                print('---  left rotor step  ---')

        print('rp = ', rgtpos, 'mp = ', midpos, 'lp = ', lftpos)
        out = full_scramble(c, lpos=lftpos, mpos=midpos, rpos=rgtpos)
        print('out:   ', out,'\n')
        encoded += out

    return encoded

In [112]:
enigmatise(word)

in:     o
rp =  25 mp =  3 lp =  11
first rotor:   O --> W
r1 to r2:      W --> A
second rotor:  A --> A
r2 to r3:      A --> I
third rotor:   I --> R
r3 to reflctr: R --> G
reflected:     G --> L
first rotor:   L --> R
r1 to r2:      R --> J
second rotor:  J --> B
r2 to r3:      B --> X
third rotor:   X --> Q
r3 back out:   Q --> R
out:    R 

in:     h
rp =  0 mp =  3 lp =  11
first rotor:   H --> Q
r1 to r2:      Q --> T
second rotor:  T --> N
r2 to r3:      N --> V
third rotor:   V --> M
r3 to reflctr: M --> B
reflected:     B --> R
first rotor:   R --> G
r1 to r2:      G --> Y
second rotor:  Y --> V
r2 to r3:      V --> S
third rotor:   S --> S
r3 back out:   S --> S
out:    S 

in:     w
rp =  1 mp =  3 lp =  11
first rotor:   W --> R
r1 to r2:      R --> T
second rotor:  T --> N
r2 to r3:      N --> V
third rotor:   V --> M
r3 to reflctr: M --> B
reflected:     B --> R
first rotor:   R --> G
r1 to r2:      G --> Y
second rotor:  Y --> V
r2 to r3:      V --> T
third rotor:   T --

r2 to r3:      P --> W
third rotor:   W --> U
r3 to reflctr: U --> I
reflected:     I --> P
first rotor:   P --> A
r1 to r2:      A --> T
second rotor:  T --> N
r2 to r3:      N --> Q
third rotor:   Q --> H
r3 back out:   H --> Z
out:    Z 

in:     f
rp =  9 mp =  5 lp =  12
first rotor:   F --> Y
r1 to r2:      Y --> U
second rotor:  U --> P
r2 to r3:      P --> W
third rotor:   W --> U
r3 to reflctr: U --> I
reflected:     I --> P
first rotor:   P --> A
r1 to r2:      A --> T
second rotor:  T --> N
r2 to r3:      N --> R
third rotor:   R --> X
r3 back out:   X --> O
out:    O 

in:     u
rp =  10 mp =  5 lp =  12
first rotor:   U --> L
r1 to r2:      L --> G
second rotor:  G --> R
r2 to r3:      R --> Y
third rotor:   Y --> Q
r3 to reflctr: Q --> E
reflected:     E --> Q
first rotor:   Q --> G
r1 to r2:      G --> Z
second rotor:  Z --> S
r2 to r3:      S --> X
third rotor:   X --> Q
r3 back out:   Q --> G
out:    G 

in:     l
rp =  11 mp =  5 lp =  12
first rotor:   L --> B
r1 to 

second rotor:  U --> P
r2 to r3:      P --> V
third rotor:   V --> M
r3 to reflctr: M --> A
reflected:     A --> Y
first rotor:   Y --> U
r1 to r2:      U --> O
second rotor:  O --> Y
r2 to r3:      Y --> K
third rotor:   K --> B
r3 back out:   B --> J
out:    J 

in:     g
rp =  19 mp =  6 lp =  12
first rotor:   G --> J
r1 to r2:      J --> W
second rotor:  W --> F
r2 to r3:      F --> L
third rotor:   L --> V
r3 to reflctr: V --> J
reflected:     J --> X
first rotor:   X --> E
r1 to r2:      E --> Y
second rotor:  Y --> V
r2 to r3:      V --> I
third rotor:   I --> V
r3 back out:   V --> C
out:    C 

in:     s
rp =  20 mp =  6 lp =  12
first rotor:   S --> O
r1 to r2:      O --> A
second rotor:  A --> A
r2 to r3:      A --> G
third rotor:   G --> C
r3 to reflctr: C --> Q
reflected:     Q --> E
first rotor:   E --> Y
r1 to r2:      Y --> S
second rotor:  S --> E
r2 to r3:      E --> S
third rotor:   S --> S
r3 back out:   S --> Y
out:    Y 

in:     g
rp =  21 mp =  6 lp =  12
first

'RSKSLMMRIOJNNGGUYAEKXMSYAYIXEIWUNKLZOGVBYEHQNAIMZVWWWGJNHGBGXGHPBEVKHEMJCYJAVRQSUGRJ'

In [113]:
enigmatise('RSKSLMMRIOJNNGGUYAEKXMSYAYIXEIWUNKLZOGVBYEHQNAIMZVWWWGJNHGBGXGHPBEVKHEMJCYJAVRQSUGRJ')

in:     R
rp =  25 mp =  3 lp =  11
first rotor:   R --> X
r1 to r2:      X --> B
second rotor:  B --> J
r2 to r3:      J --> R
third rotor:   R --> W
r3 to reflctr: W --> L
reflected:     L --> G
first rotor:   G --> I
r1 to r2:      I --> A
second rotor:  A --> A
r2 to r3:      A --> W
third rotor:   W --> N
r3 back out:   N --> O
out:    O 

in:     S
rp =  0 mp =  3 lp =  11
first rotor:   S --> S
r1 to r2:      S --> V
second rotor:  V --> Y
r2 to r3:      Y --> G
third rotor:   G --> C
r3 to reflctr: C --> R
reflected:     R --> B
first rotor:   B --> V
r1 to r2:      V --> N
second rotor:  N --> T
r2 to r3:      T --> Q
third rotor:   Q --> H
r3 back out:   H --> H
out:    H 

in:     K
rp =  1 mp =  3 lp =  11
first rotor:   K --> T
r1 to r2:      T --> V
second rotor:  V --> Y
r2 to r3:      Y --> G
third rotor:   G --> C
r3 to reflctr: C --> R
reflected:     R --> B
first rotor:   B --> V
r1 to r2:      V --> N
second rotor:  N --> T
r2 to r3:      T --> R
third rotor:   R --

first rotor:   G --> X
r1 to r2:      X --> S
second rotor:  S --> Z
r2 to r3:      Z --> G
third rotor:   G --> C
r3 to reflctr: C --> Q
reflected:     Q --> E
first rotor:   E --> Y
r1 to r2:      Y --> R
second rotor:  R --> G
r2 to r3:      G --> L
third rotor:   L --> E
r3 back out:   E --> U
out:    U 

in:     V
rp =  11 mp =  5 lp =  12
first rotor:   V --> D
r1 to r2:      D --> X
second rotor:  X --> V
r2 to r3:      V --> C
third rotor:   C --> F
r3 to reflctr: F --> T
reflected:     T --> Z
first rotor:   Z --> F
r1 to r2:      F --> Y
second rotor:  Y --> V
r2 to r3:      V --> B
third rotor:   B --> W
r3 back out:   W --> L
out:    L 

in:     B
rp =  12 mp =  5 lp =  12
first rotor:   B --> W
r1 to r2:      W --> P
second rotor:  P --> C
r2 to r3:      C --> J
third rotor:   J --> T
r3 to reflctr: T --> H
reflected:     H --> D
first rotor:   D --> H
r1 to r2:      H --> A
second rotor:  A --> A
r2 to r3:      A --> H
third rotor:   H --> P
r3 back out:   P --> D
out:   

r2 to r3:      P --> G
third rotor:   G --> F
r3 back out:   F --> I
out:    I 

in:     R
rp =  24 mp =  6 lp =  12
first rotor:   R --> H
r1 to r2:      H --> P
second rotor:  P --> C
r2 to r3:      C --> I
third rotor:   I --> R
r3 to reflctr: R --> F
reflected:     F --> S
first rotor:   S --> P
r1 to r2:      P --> J
second rotor:  J --> B
r2 to r3:      B --> T
third rotor:   T --> L
r3 back out:   L --> N
out:    N 

in:     Q
rp =  25 mp =  6 lp =  12
first rotor:   Q --> H
r1 to r2:      H --> O
second rotor:  O --> M
r2 to r3:      M --> S
third rotor:   S --> G
r3 to reflctr: G --> U
reflected:     U --> C
first rotor:   C --> Z
r1 to r2:      Z --> T
second rotor:  T --> N
r2 to r3:      N --> G
third rotor:   G --> F
r3 back out:   F --> G
out:    G 

in:     S
rp =  0 mp =  6 lp =  12
first rotor:   S --> S
r1 to r2:      S --> Y
second rotor:  Y --> O
r2 to r3:      O --> U
third rotor:   U --> K
r3 to reflctr: K --> Y
reflected:     Y --> A
first rotor:   A --> V
r1 to 

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'